In [84]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import pickle as pkl
from input_output import read_csv, write_csv
from utils import get_df_info, get_current_time
from bs4 import BeautifulSoup
import codecs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from functools import reduce

logging.getLogger('matplotlib').setLevel(logging.ERROR)

logger = logging.getLogger(__name__)

if not os.path.exists('./logs'):
    os.mkdir('./logs')

In [85]:
current_time = get_current_time()
f_hdlr = logging.FileHandler('./logs/{}.log'.format(current_time))
fmt = '%(asctime)s [%(levelname)s]: %(name)s - %(message)s'
hdlr = [f_hdlr, logging.StreamHandler()]
logging.basicConfig(level=logging.DEBUG, format=fmt, handlers=hdlr)

np.random.seed(42)

path = './data/FINANCIALNEWS_sample.csv'
df = read_csv(path)

df = df.set_index('OBJECT_ID')

print(df.head(10))
get_df_info(df)

Current Time = 2020-04-26_22:28:33


2020-04-26 22:28:33,606 [INFO]: input_output - Read 1000 rows and 15 cols from ./data/FINANCIALNEWS_sample.csv.


                  PUBLISHDATE                                      TITLE  \
OBJECT_ID                                                                  
379746418  2019-4-23 13:26:52                    公募基金二季报重仓股全揭露，8类排名一文打尽！   
453849759  2019-1-25 19:02:01                         民航局：将开展无人机适航审定分级管理   
453856916  2019-1-25 20:42:35                 民航局：推动形成基于运行风险的民用无人机适航管理模式   
453941181  2019-1-28 00:53:00                促进民营经济发展 14地设纾困基金 总规模超1330亿   
453948030  2019-1-28 07:55:16               国资委鼓励外资参与国企改革，业内人士建议做好资本进出安排   
453949829  2019-1-28 08:13:16                   趣墅时隔一年再获千万融资，别墅度假领域战火重燃？   
453954720  2019-1-28 09:06:18                  融资客青睐28股，豫能控股买入占比高达43.91%   
453955303  2019-1-28 09:12:13                    2018年哈萨克斯坦铁合金产量同比增长7.9%   
453956430  2019-1-28 09:22:45  中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等   
453957310  2019-1-28 09:32:15                            蓝色起源开始建厂生产火箭发动机   

                                                     CONTENT SOURCE  \
OBJECT_ID       

In [86]:
def get_text(c):
    soup = BeautifulSoup(c, 'html.parser')
    return soup.getText()

In [4]:
df['clean_text'] = df['CONTENT'].apply(lambda x: get_text(x))

In [5]:
df.WINDCODES

,PUBLISHDATE,TITLE,CONTENT,SOURCE,URL,SECTIONS,AREACODES,WINDCODES,INDUSTRYCODES,SECTERCODES,KEYWORDS,OPDATE,OPMODE,MKTSENTIMENTS,clean_text
OBJECT_ID,,,,,,,,,,,,,,,
379746418,2019-4-23 13:26:52,公募基金二季报重仓股全揭露，8类排名一文打尽！,"<P style=""MARGIN-BOTTOM: 1.6em; MARGIN-TOP: 0p...",Wind,http://news.windin.com/ns/findsnap.php?sourcet...,人工新闻|基金|标准新闻|旧版新闻|精选新闻|分类,NaN,NaN,NaN,NaN,NaN,2019-4-23 13:32:43,0,NaN,香港万得通讯社报道，随着7月21日基金二季报披露的最后节点过去，目前基金重仓股已经全部显露，...
453849759,2019-1-25 19:02:01,民航局：将开展无人机适航审定分级管理,"<DIV><DIV><P style="";text-align:justify"">据民航局网...",证券时报网,http://kuaixun.stcn.com/2019/0125/14825024.shtml,三板|旧版新闻|精选新闻|分类|宏观政策模型识别|功能关键字|热点专题,NaN,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,882434.WI:航空货运与物流|882215.WI:航空货运与物流Ⅲ|882104.WI...,NaN,NaN,2019-2-1 11:15:53,0,NaN,据民航局网站25日消息， 为促进国内民用无人机产业健康成长，1月23日，民航局印发《基于运行...
453856916,2019-1-25 20:42:35,民航局：推动形成基于运行风险的民用无人机适航管理模式,"<DIV><DIV><P>e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导...",e公司,http://www.egsea.com/news/detail?id=356159,三板|旧版新闻|精选新闻|分类|宏观政策模型识别|功能关键字|热点专题,NaN,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,882434.WI:航空货运与物流|882215.WI:航空货运与物流Ⅲ|882104.WI...,NaN,NaN,2019-2-1 11:15:53,0,NaN,"e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导意见》，针对无人机运行场景..."
453941181,2019-1-28 00:53:00,促进民营经济发展 14地设纾困基金 总规模超1330亿,<DIV><DIV><DIV><P><SPAN>多省市出台措施意见促进民营经济发展 明确对暂...,北青网,https://finance.sina.com.cn/roll/2019-01-28/do...,基金|标准新闻|精选新闻|分类|旧版新闻|宏观政策模型识别|功能关键字|热点专题,NaN,000011.SH:上证基金|ON0209:沪深|399103.SZ:乐富|885010.W...,NaN,NaN,NaN,2019-2-2 11:07:30,0,NaN,多省市出台措施意见促进民营经济发展 明确对暂时困难企业稳贷、续贷14地设纾困基金 总规模超1...
453948030,2019-1-28 07:55:16,国资委鼓励外资参与国企改革，业内人士建议做好资本进出安排,"<!--USED_TEMPLATE:无图新闻模板--><P style=""MARGIN-BO...",Wind,http://snap.windin.com/ns/findsnap.php?sourcet...,宏观|国内财经|标准新闻|旧版新闻|功能关键字|直播|分类|人工新闻|精选新闻|热点专题,DQXW01:大中华区|DQXW:地区新闻|DQXW0101:中国|ON05:地区,NaN,NaN,NaN,NaN,2019-2-2 09:32:23,0,NaN,证券日报报道，近日，国资委会同11家中央企业赴瑞士达沃斯出席世界经济论坛2019年年会有关活...
453949829,2019-1-28 08:13:16,趣墅时隔一年再获千万融资，别墅度假领域战火重燃？,"<DIV><DIV><P style=""""><STRONG>【环球旅讯】</strong>成...",环球旅讯,https://www.traveldaily.cn/article/127156,精选新闻|旧版新闻|分类|宏观政策模型识别|功能关键字,NaN,NaN,6225301020:酒店、度假村与豪华游轮|6225301000:酒店、餐馆与休闲Ⅲ|62...,NaN,NaN,2019-2-4 06:18:00,0,NaN,【环球旅讯】成立于2016年1月的趣墅，是一家通过与地产商及别墅业主合作运营别墅度假村、整合...
453954720,2019-1-28 09:06:18,融资客青睐28股，豫能控股买入占比高达43.91%,"<!--USED_TEMPLATE:无图新闻模板--><P style=""MARGIN-BO...",Wind,http://snap.windin.com/ns/findsnap.php?sourcet...,股市|个股|标准新闻|旧版新闻|个股|A股|Wind资讯|功能关键字|直播|精选新闻|自动新...,DQXW:地区新闻|DQXW01:大中华区|DQXW0101:中国|ON05:地区,300113.SZ:顺网科技|002073.SZ:软控股份|000989.SZ:九芝堂|00...,NaN,NaN,NaN,2019-2-1 09:11:39,0,NaN,香港万得通讯社报道，1月25日，Wind数据统计显示，截至收盘，沪深两市共有947只个股有融...
453955303,2019-1-28 09:12:13,2018年哈萨克斯坦铁合金产量同比增长7.9%,<DIV><DIV><DIV><SPAN>\t\t 阿斯塔纳1月16日消息，哈萨克斯坦统计...,兰格钢铁网,http://luliao.lgmi.com/html/201901/28/7251.htm,精选新闻|旧版新闻|分类,NaN,NaN,6215104050:钢铁|6215104000:金属、非金属与采矿|6215100000:...,NaN,NaN,2019-2-4 06:28:05,0,NaN,\t\t 阿斯塔纳1月16日消息，哈萨克斯坦统计委员会周三公布的数据显示，2018年12月...
453956430,2019-1-28 09:22:45,中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等,"<DIV><DIV><P style=""margin-top:5px;border-radi...",微信,http://mp.weixin.qq.com/s?__biz=MzAxMzAyMzI4NQ...,外汇|标准新闻|外汇|旧版新闻|股票名家|功能关键字|个股|海外中国股|股市|精选新闻|分类,DQXW0101:中国|DQXW01:大中华区|DQXW:地区新闻|DQ0001:中国（临时...,000860.SZ:顺鑫农业|ON0201:A股|EURCNY.EX:欧元中间价|ON023...,NaN,NaN,NaN,2019-2-4 06:34:07,0,NaN,目 录宏观策略【策略】基金四季度持仓报告：重仓股集中度下降，是否是新趋势的开始【海外】1月...


In [6]:
df = df[(~df.clean_text.str.contains('您没有浏览')) & (~df.WINDCODES.isnull())][['TITLE','WINDCODES','clean_text']]

In [7]:
df['clean_codes'] = df['WINDCODES'].apply(lambda x: x.split('|')).apply(lambda x: [i.split(':')[0] for i in x])

In [8]:
df['n_codes'] = df.clean_codes.apply(lambda x: len(x))

In [83]:
df

,TITLE,WINDCODES,clean_text,clean_codes,n_codes
OBJECT_ID,,,,,
453849759,民航局：将开展无人机适航审定分级管理,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,据民航局网站25日消息， 为促进国内民用无人机产业健康成长，1月23日，民航局印发《基于运行...,"[832201.OC, ON0207, h11043.CSI, ON0209, ON02]",5
453856916,民航局：推动形成基于运行风险的民用无人机适航管理模式,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,"e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导意见》，针对无人机运行场景...","[832201.OC, ON0207, h11043.CSI, ON0209, ON02]",5
453941181,促进民营经济发展 14地设纾困基金 总规模超1330亿,000011.SH:上证基金|ON0209:沪深|399103.SZ:乐富|885010.W...,多省市出台措施意见促进民营经济发展 明确对暂时困难企业稳贷、续贷14地设纾困基金 总规模超1...,"[000011.SH, ON0209, 399103.SZ, 885010.WI, ON02]",5
453954720,融资客青睐28股，豫能控股买入占比高达43.91%,300113.SZ:顺网科技|002073.SZ:软控股份|000989.SZ:九芝堂|00...,香港万得通讯社报道，1月25日，Wind数据统计显示，截至收盘，沪深两市共有947只个股有融...,"[300113.SZ, 002073.SZ, 000989.SZ, 002340.SZ, 0...",21
453956430,中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等,000860.SZ:顺鑫农业|ON0201:A股|EURCNY.EX:欧元中间价|ON023...,目 录宏观策略【策略】基金四季度持仓报告：重仓股集中度下降，是否是新趋势的开始【海外】1月...,"[000860.SZ, ON0201, EURCNY.EX, ON0236, EURUSD....",8
453957310,蓝色起源开始建厂生产火箭发动机,744572:BE|3783:公司实体|ON02:公司,PingWest品玩1月28日报道，蓝色起源已正式开始在美国阿拉巴马州亨茨维尔建厂，为新格伦...,"[744572, 3783, ON02]",3
453958250,人工智能，在中国表现如何？,600794:清华大学|3783:公司实体|ON02:公司,人工智能的发展已经是不可逆转的趋势，各国都在争相布局这一高新科技产业，中国也不例外。作者 |...,"[600794, 3783, ON02]",3
453959732,富阳新兴纸业有限公司等三企业核心资产司法拍卖公告,h30049.CSI:AMAC造纸|ON0209:沪深|ON02:公司,杭州富阳新兴纸业有限公司、杭州新兴纸业有限公司、杭州新沃无纺布有限公司名下位于富阳区大源镇新...,"[h30049.CSI, ON0209, ON02]",3
453962765,湖北百亿纾困基金蓄势待发 国企券商多头援上市公司,ON0201:A股|0218.HK:申万宏源|ON0204:港股|H01085.HK:申万宏...,国企、券商多头并进驰援上市公司图文：湖北百亿纾困基金蓄势待发楚天都市报为缓解湖北上市企业阶段...,"[ON0201, 0218.HK, ON0204, H01085.HK, jMhn1VeEU...",7


In [77]:
clean_codes = np.array(reduce(lambda x, y: x+y, df['clean_codes'])).reshape(-1,1)

In [74]:
obj = df.index.repeat(df.n_codes).values.reshape(-1,1)

In [80]:
arr = np.hstack([clean_codes, obj])
obj_df = pd.DataFrame(arr, columns=['code', 'obj'])

In [107]:
uniq_codes = pd.Series(obj_df.code.unique())
sus_codes = uniq_codes[~(uniq_codes.str.endswith('SH'))& (~uniq_codes.str.endswith('SZ'))]
good_codes = uniq_codes[(uniq_codes.str.endswith('SH')) | (uniq_codes.str.endswith('SZ'))]
pd.DataFrame(sus_codes.values).to_csv('unknown_windcodes.csv', index=False)

In [110]:
good_codes

5       000011.SH
6       399103.SZ
8       300113.SZ
9       002073.SZ
10      000989.SZ
11      002340.SZ
12      002071.SZ
13      600267.SH
14      600502.SH
15      000899.SZ
16      600655.SH
17      603001.SH
18      000551.SZ
19      600976.SH
20      600284.SH
21      002117.SZ
22      000001.SZ
23      000750.SZ
24      002369.SZ
25      600677.SH
26      002460.SZ
28      000860.SZ
60      000564.SZ
61      600280.SH
66      601155.SH
71      601901.SH
72      002450.SZ
74      002102.SZ
76      002259.SZ
78      300362.SZ
          ...    
1619    002417.SZ
1620    600084.SH
1621    000929.SZ
1622    603707.SH
1623    603517.SH
1625    000608.SZ
1679    600027.SH
1681    002011.SZ
1685    002668.SZ
1694    300090.SZ
1700    600837.SH
1701    600052.SH
1704    002583.SZ
1705    002900.SZ
1710    000623.SZ
1711    601006.SH
1761    002947.SZ
1762    603956.SH
1763    002949.SZ
1764    300758.SZ
1765    600928.SH
1786    510353.SH
1798    160925.SZ
1800    600616.SH
1804    60

In [6]:
stopwords = codecs.open('data/chinese_stopwords.txt', 'r', 'utf-8').read().split(',')
stopwords = [x.replace('\n','').strip() for x in stopwords]

In [7]:
stopwords

['按',
 '按照',
 '俺',
 '俺们',
 '阿别',
 '别人',
 '别处',
 '别是',
 '别的',
 '别管',
 '别说',
 '不',
 '不仅',
 '不但',
 '不光',
 '不单',
 '不只',
 '不外乎',
 '不如',
 '不妨',
 '不尽',
 '不尽然',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不料',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不至于',
 '不若',
 '不论',
 '不过',
 '不问',
 '比方',
 '比如',
 '比及',
 '比',
 '本身',
 '本着',
 '本地',
 '本人',
 '本',
 '巴巴',
 '巴',
 '并',
 '并且',
 '非彼',
 '彼时',
 '彼此',
 '便于',
 '把',
 '边',
 '鄙人',
 '罢了',
 '被',
 '般的此间',
 '此次',
 '此时',
 '此外',
 '此处',
 '此地',
 '此',
 '才',
 '才能',
 '朝',
 '朝着',
 '从',
 '从此',
 '从而',
 '除非',
 '除此之外',
 '除开',
 '除外',
 '除了',
 '除',
 '诚然',
 '诚如',
 '出来',
 '出于',
 '曾',
 '趁着',
 '趁',
 '处在',
 '乘',
 '冲等等',
 '等到',
 '等',
 '第',
 '当着',
 '当然',
 '当地',
 '当',
 '多',
 '多么',
 '多少',
 '对',
 '对于',
 '对待',
 '对方',
 '对比',
 '得',
 '得了',
 '打',
 '打从',
 '的',
 '的确',
 '的话',
 '但',
 '但凡',
 '但是',
 '大家',
 '大',
 '地',
 '待',
 '都',
 '到',
 '叮咚而言',
 '而是',
 '而已',
 '而外',
 '而后',
 '而况',
 '而且',
 '而',
 '尔尔',
 '尔后',
 '尔',
 '二来非独',
 '非特',
 '非徒',
 '非但',
 '否则',
 '反过来说',
 '反过来',
 '反而',
 '反之',
 '分别',
 '凡是',
 '凡',
 '个',

In [28]:
def get_LSA(documents, nc):
    vectorizer = TfidfVectorizer(use_idf=True, 
                                 smooth_idf=True,
                                 ngram_range=(1,2))
    svd_model = TruncatedSVD(n_components=nc,
                             algorithm='randomized',
                             n_iter=10)
    svd_transformer = Pipeline([('tfidf', vectorizer), 
                                ('svd', svd_model)])
    svd_matrix = svd_transformer.fit_transform(documents)
    return svd_transformer, svd_matrix

In [31]:
svd_transformer, svd_matrix = get_LSA(df.clean_text, 10)

In [32]:
terms = svd_transformer['tfidf'].get_feature_names()

for i, comp in enumerate(svd_transformer['svd'].components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+ str(i) + ": ")
    print(sorted_terms)
    ts = ' '.join([t[0] for t in sorted_terms])
    print(ts)

Topic 0: 
[('如需浏览新闻全文', 0.3789246883542835), ('如需浏览新闻全文 请联系您的客户经理', 0.3789246883542835), ('您没有浏览此新闻全文的权限', 0.3789246883542835), ('您没有浏览此新闻全文的权限 如需浏览新闻全文', 0.3789246883542835), ('请联系您的客户经理', 0.3789246883542835), ('请联系您的客户经理 谢谢', 0.3789246883542835), ('谢谢', 0.3721432423611532)]
如需浏览新闻全文 如需浏览新闻全文 请联系您的客户经理 您没有浏览此新闻全文的权限 您没有浏览此新闻全文的权限 如需浏览新闻全文 请联系您的客户经理 请联系您的客户经理 谢谢 谢谢
Topic 1: 
[('换手率达20', 0.32105198329821344), ('00', 0.2630027174241469), ('平仓线', 0.1371926582374725), ('不得低于130', 0.12739182119167086), ('平仓线 不得低于130', 0.12739182119167086), ('的统一限制', 0.12252699723241832), ('涨跌幅偏离值达7', 0.11892380482292136)]
换手率达20 00 平仓线 不得低于130 平仓线 不得低于130 的统一限制 涨跌幅偏离值达7
Topic 2: 
[('换手率达20', 0.4728133584868632), ('00', 0.3570178518281692), ('涨跌幅偏离值达7', 0.17173240341806448), ('万元占总成交比例净额', 0.1392680716567453), ('01', 0.0729803521003407), ('万元占总成交比例买入金额', 0.06963403582837265), ('万元占总成交比例买入金额 万元占总成交比例净额', 0.06963403582837265)]
换手率达20 00 涨跌幅偏离值达7 万元占总成交比例净额 01 万元占总成交比例买入金额 万元占总成交比例买入金额 万元占总成交比例净额
Topic 3: 
[('万